In [3]:
import tensorflow as tf
import tensorflow_hub as hub
!pip install tensorflow_text
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 45.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
df = pd.read_csv("spam.csv")
df.head(5) #stampa primi 5

In [ ]:
df.groupby('Category').describe() #fa vedere quante righe appartengono alla stessa categoria

In [ ]:
#since the dataset is unbalanced we have to set it up --> downsampling (non il massimo ma per questo esempio va bene)

df.groupby('Category').vale_counts()

In [ ]:
747/4825 #0.1548 

In [ ]:
df_spam = df[df['Category']=='spam']
df_spam.shape

In [ ]:
df_ham = df[df['Category']=='ham']
df_ham.shape

In [ ]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape 

In [ ]:
df_balanced = pd.concat([df_spam,df_ham_downsampled])
df_balanced.shape
df_balanced.groupby('Category').vale_counts()

In [ ]:
df_balanced['spam'] = df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)



In [ ]:
from sklearn.model_selection import train_test_split

x_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'], df_balanced['spam'], stratify = df_balanced['spam'])

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")


In [ ]:
def get_sentence_embedding(sentences):
  preprocessed_text = bert_preprocess(sentences)
  return bert_encoder(preprocessed_text)['pooled_output']
  

In [ ]:
e = get_sentence_embedding([
    "banana",
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([e[0]],[e[1]])

In [ ]:
# Bert layers

 text_input = tf.keras.layers.Input(shape=(),dtype=tf.string, name="text")

 preprocessed_text = bert_preprocess(text_input)

 outputs = bert_encoder(preprocessed_text)

# NN Layers

l = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])

l= tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)

#construct final model
model = tf.keras.Model(inputs=[text_input], outputs=[l])

In [ ]:
model.summary()

In [ ]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [ ]:
import numpy as np
y_predicted = np.where(y_predicted >0.5,1,0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test,y_predicted)
cm

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm,annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_predicted))

Inference